### Data Mining and Machine Learning
### Ensembles of classifiers
#### Datasets:  Diabetes and Landsat
#### Edgar Acuna
#### April 2020

In [4]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
h2o.init(ip="localhost", port=54323)
#h2o.no_progress()

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O cluster uptime:,2 mins 45 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.27.0.4835
H2O cluster version age:,5 months and 8 days !!!
H2O cluster name:,H2O_from_python_eacun_msgzj3
H2O cluster total nodes:,1
H2O cluster free memory:,3.518 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


### Bootstrap Samples

In [5]:
#This is the orginal training sample L 
x=[5,3,12,13,21,31,8,9,15,17,24,32] 

In [6]:
#This is bootsrap sample(sample with replacement)
boot1=np.random.choice(x,12)
print(boot1)

[17 32 15 32 12  5 17 15 31 17  5  5]


In [7]:
np.unique(boot1)

array([ 5, 12, 15, 17, 31, 32])

In [8]:
#another boostrapp sample
boot2=np.random.choice(x,12)
print(boot2)

[15  9  3 21  5  5 15 15 15  9 15 17]


In [6]:
np.unique(boot2)

array([ 3,  8,  9, 12, 13, 15, 17, 21, 24, 32])

Note: Approximately 37% of the  instances of the training sample  L DO NOT appear in any bootstrap sample. In the above examples 16.67% and 41.67% of instances do not appear in each  of the bootstrap samples.

### I. Bagging for Diabetes using trees and scikit learn

In [9]:
url= "http://academic.uprm.edu/eacuna/diabetes.dat"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_table(url, names=names,header=None)
#The response varaiable must be binary  (0,1)
y=data['class']-1
X=data.iloc[:,0:8]
modeltree = tree.DecisionTreeClassifier()
bagging = BaggingClassifier(modeltree,n_estimators=100)

In [10]:
# Accuracy rate by resubstitution
bagging.fit(X, y)
predictions = bagging.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       268

    accuracy                           1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768



In [11]:
#Estimating the accuracy by cross validation
kfold = model_selection.KFold(n_splits=10, random_state=99)
results = model_selection.cross_val_score(bagging, X, y, cv=kfold)
print(results.mean())

0.7603383458646615


#### Out-of-Bag accuracy

In [12]:
bagging1 = BaggingClassifier(modeltree,n_estimators=50, oob_score=True)
bagging1.fit(X, y)
bagging1.oob_score_

0.7486979166666666

### II. AdaBoosting para Diabetes usando scikit-learn

In [13]:
adaboost = AdaBoostClassifier(modeltree,n_estimators=100,learning_rate=1)
adaboost.fit(X, y)
predictions = adaboost.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       268

    accuracy                           1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768



In [14]:
#Estimating the accuracy by cross-validation
kfold = model_selection.KFold(n_splits=10, random_state=999)
results = model_selection.cross_val_score(adaboost, X, y, cv=kfold)
print(results.mean())

0.6991285030758715


### III. Gradient Boosting para Diabetes usando scikit-learn

In [16]:
gboost = GradientBoostingClassifier(n_estimators=100)
#X_train, X_train_lr, y_train, y_train_lr = train_test_split(X,y,test_size=0.5)
gboost.fit(X, y)
predictions = gboost.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       500
           1       0.91      0.81      0.86       268

    accuracy                           0.91       768
   macro avg       0.91      0.88      0.89       768
weighted avg       0.91      0.91      0.90       768



In [17]:
#Estimating the accuracy by cross-validation
kfold = model_selection.KFold(n_splits=10, random_state=999)
results = model_selection.cross_val_score(gboost, X, y, cv=kfold)
print(results.mean())

0.7669002050580999


### IV  Gradient Boosting for diabetes using h2o

In [19]:
diabetes = h2o.import_file("https://academic.uprm.edu/eacuna/diabetes.dat")
myx=['C1','C2','C3','C4','C5','C6','C7','C8']
diabetes['C9']=diabetes['C9'].asfactor()
myy="C9"
gbm1 = H2OGradientBoostingEstimator(model_id="gbm_covType_v1",ntrees = 100, max_depth=4,nfolds=10, sample_rate = 1,col_sample_rate = 1,seed=20000)
gbm1.train(myx, myy, training_frame=diabetes)
y_pred=gbm1.predict(diabetes)
print((y_pred['predict']==diabetes['C9']).mean())

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
[0.9388020833333334]


In [20]:
#Accuracy ny resubstitution
gbm1.model_performance(diabetes)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.06189488461506235
RMSE: 0.24878682564609877
LogLoss: 0.2278218904572922
Mean Per-Class Error: 0.06844776119402984
AUC: 0.9836082089552238
pr_auc: 0.9663482460483861
Gini: 0.9672164179104477

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4582730505624667: 


,,1,2,Error,Rate
0,1,482.0,18.0,0.036,(18.0/500.0)
1,2,28.0,240.0,0.1045,(28.0/268.0)
2,Total,510.0,258.0,0.0599,(46.0/768.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.458273,0.912548,171.0
1,max f2,0.333183,0.929140,206.0
2,max f0point5,0.538282,0.936441,148.0
3,max accuracy,0.458273,0.940104,171.0
4,max precision,0.985922,1.000000,0.0
5,max recall,0.104561,1.000000,309.0
6,max specificity,0.985922,1.000000,0.0
7,max absolute_mcc,0.458273,0.867390,171.0
8,max min_per_class_accuracy,0.412628,0.929104,185.0
9,max mean_per_class_accuracy,0.412628,0.931552,185.0



Gains/Lift Table: Avg response rate: 34.90 %, avg score: 34.90 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010417,0.974314,2.865672,2.865672,1.000000,0.979208,1.000000,0.979208,0.029851,0.029851,186.567164,186.567164
1,,2,0.020833,0.964821,2.865672,2.865672,1.000000,0.969928,1.000000,0.974568,0.029851,0.059701,186.567164,186.567164
2,,3,0.031250,0.955925,2.865672,2.865672,1.000000,0.960038,1.000000,0.969725,0.029851,0.089552,186.567164,186.567164
3,,4,0.040365,0.952782,2.865672,2.865672,1.000000,0.954008,1.000000,0.966176,0.026119,0.115672,186.567164,186.567164
4,,5,0.050781,0.941174,2.865672,2.865672,1.000000,0.946141,1.000000,0.962066,0.029851,0.145522,186.567164,186.567164
5,,6,0.100260,0.897921,2.865672,2.865672,1.000000,0.919839,1.000000,0.941227,0.141791,0.287313,186.567164,186.567164
6,,7,0.151042,0.835725,2.865672,2.865672,1.000000,0.869110,1.000000,0.916981,0.145522,0.432836,186.567164,186.567164
7,,8,0.200521,0.752943,2.865672,2.865672,1.000000,0.789735,1.000000,0.885582,0.141791,0.574627,186.567164,186.567164
8,,9,0.300781,0.532979,2.567940,2.766428,0.896104,0.648036,0.965368,0.806400,0.257463,0.832090,156.793952,176.642760
9,,10,0.399740,0.345884,1.206599,2.380281,0.421053,0.436178,0.830619,0.714749,0.119403,0.951493,20.659859,138.028101


In [22]:
#Mostrando la matrix de confusion para estimar la precision out-of-bag y por validacion crizada
gbm1.confusion_matrix

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_covType_v1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,100.0,100.0,19367.0,4.0,4.0,4.0,5.0,16.0,10.74




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.06189488480582112
RMSE: 0.24878682602947674
LogLoss: 0.2278218932658851
Mean Per-Class Error: 0.06844776119402984
AUC: 0.9836082089552238
pr_auc: 0.9663482460483861
Gini: 0.9672164179104477

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4582730550290878: 


,,1,2,Error,Rate
0,1,482.0,18.0,0.036,(18.0/500.0)
1,2,28.0,240.0,0.1045,(28.0/268.0)
2,Total,510.0,258.0,0.0599,(46.0/768.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-04-06 16:20:30,6.466 sec,0.0,0.476641,0.646799,0.500000,0.000000,1.000000,0.651042
1,,2020-04-06 16:20:30,6.484 sec,1.0,0.458367,0.609445,0.870556,0.584042,2.626866,0.218750
2,,2020-04-06 16:20:30,6.492 sec,2.0,0.443037,0.579248,0.875873,0.611844,2.703464,0.213542
3,,2020-04-06 16:20:30,6.504 sec,3.0,0.430069,0.554210,0.882642,0.616080,2.703464,0.208333
4,,2020-04-06 16:20:30,6.508 sec,4.0,0.419352,0.533754,0.883937,0.642541,2.800543,0.205729
5,,2020-04-06 16:20:30,6.516 sec,5.0,0.409897,0.515609,0.886638,0.682676,2.865672,0.217448
6,,2020-04-06 16:20:30,6.524 sec,6.0,0.400978,0.498875,0.894974,0.720901,2.865672,0.187500
7,,2020-04-06 16:20:30,6.533 sec,7.0,0.393181,0.484229,0.900034,0.730095,2.865672,0.184896
8,,2020-04-06 16:20:30,6.538 sec,8.0,0.386140,0.470769,0.903716,0.745706,2.865672,0.187500
9,,2020-04-06 16:20:30,6.545 sec,9.0,0.379840,0.458879,0.906813,0.770021,2.865672,0.171875



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,C2,253.061905,1.000000,0.401102
1,C6,133.097443,0.525948,0.210959
2,C8,78.790070,0.311347,0.124882
3,C7,65.523659,0.258923,0.103855
4,C1,35.476620,0.140189,0.056230
5,C5,30.243166,0.119509,0.047935
6,C3,22.359510,0.088356,0.035440
7,C4,12.364817,0.048861,0.019598


<bound method H2OBinomialModel.confusion_matrix of >

### V. Bagging  using Decision Trees for Landsat (scikit-learn)

In [21]:
url='http://academic.uprm.edu/eacuna/landsat.txt'
data = pd.read_table(url, header=None,delim_whitespace=True)
y=data.iloc[:,36]-1
names=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13',
            'C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','C27',
           'C28','C29', 'C30','C31','C32','C33','C34','C35','C36','C37']
X=data.iloc[:,0:36]
modeltree = tree.DecisionTreeClassifier()
bagging = BaggingClassifier(modeltree,n_estimators=100, max_features=1.0)
# Tasa de precision
bagging.fit(X, y)
predictions = bagging.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1072
           1       1.00      1.00      1.00       479
           2       1.00      1.00      1.00       961
           3       1.00      1.00      1.00       415
           4       1.00      1.00      1.00       470
           5       1.00      1.00      1.00      1038

    accuracy                           1.00      4435
   macro avg       1.00      1.00      1.00      4435
weighted avg       1.00      1.00      1.00      4435



In [19]:
#Accuracy by resubstitution
kfold = model_selection.KFold(n_splits=10, random_state=99)
results = model_selection.cross_val_score(modeltree, X, y, cv=kfold)
print(results.mean())

0.8027159213389462


In [20]:
#accuracy by out-of-bag
bagging1 = BaggingClassifier(modeltree,n_estimators=50, oob_score=True)
bagging1.fit(X, y)
bagging1.oob_score_

0.9028184892897407

### VI. AdaBoosting for Landsat

In [21]:
adaboost = AdaBoostClassifier(modeltree,n_estimators=100,learning_rate=1)
adaboost.fit(X, y)
predictions = adaboost.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1072
           1       1.00      1.00      1.00       479
           2       1.00      1.00      1.00       961
           3       1.00      1.00      1.00       415
           4       1.00      1.00      1.00       470
           5       1.00      1.00      1.00      1038

    accuracy                           1.00      4435
   macro avg       1.00      1.00      1.00      4435
weighted avg       1.00      1.00      1.00      4435



In [22]:
#accuracy by cross-validation
kfold = model_selection.KFold(n_splits=10, random_state=999)
results = model_selection.cross_val_score(adaboost, X, y, cv=kfold)
print(results.mean())

0.8049717324548024


In [23]:
gboost = GradientBoostingClassifier(n_estimators=100)
#X_train, X_train_lr, y_train, y_train_lr = train_test_split(X,y,test_size=0.5)
gboost.fit(X, y)
predictions = gboost.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1072
           1       1.00      1.00      1.00       479
           2       0.96      0.99      0.98       961
           3       0.96      0.87      0.92       415
           4       0.99      0.99      0.99       470
           5       0.98      0.98      0.98      1038

    accuracy                           0.98      4435
   macro avg       0.98      0.97      0.98      4435
weighted avg       0.98      0.98      0.98      4435



In [24]:
#Estimating the accueacy bt cross-validation
kfold = model_selection.KFold(n_splits=10, random_state=999)
results = model_selection.cross_val_score(gboost, X, y, cv=kfold)
print(results.mean())

0.8647362373660343


### VII. Gradient Boostimg for Landsat using H2o

In [25]:
#Leyendo los datos
datos= h2o.import_file("http://academic.uprm.edu/eacuna/landsat.txt")
myx=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13',
            'C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','C27',
           'C28','C29', 'C30','C31','C32','C33','C34','C35','C36']
datos['C37']=datos['C37'].asfactor()
myy="C37"
gbm2 = H2OGradientBoostingEstimator(model_id="gbm_covType_v1",ntrees = 100, max_depth=4,nfolds=10, sample_rate = 1,col_sample_rate = 1,seed=20000)
gbm2.train(myx, myy, training_frame=datos)
y_pred=gbm2.predict(datos)
print((y_pred['predict']==datos['C37']).mean())

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
[0.9950394588500564]


In [26]:
#Mostrando la matrix de confusion para estimar la precision out-of-bag y por validacion crizada
gbm2.confusion_matrix

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_covType_v1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,100.0,600.0,135530.0,4.0,4.0,4.0,7.0,16.0,13.368333




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.007986146376090133
RMSE: 0.08936524143138726
LogLoss: 0.04483049751656768
Mean Per-Class Error: 0.007389093948046522

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,1,2,3,4,5,6,Error,Rate
0,1072.0,0.0,0.0,0.0,0.0,0.0,0.000000,"0 / 1,072"
1,0.0,479.0,0.0,0.0,0.0,0.0,0.000000,0 / 479
2,0.0,0.0,961.0,0.0,0.0,0.0,0.000000,0 / 961
3,0.0,0.0,7.0,399.0,0.0,9.0,0.038554,16 / 415
4,0.0,0.0,0.0,0.0,470.0,0.0,0.000000,0 / 470
5,0.0,0.0,4.0,2.0,0.0,1032.0,0.005780,"6 / 1,038"
6,1072.0,479.0,972.0,401.0,470.0,1041.0,0.004961,"22 / 4,435"



Top-6 Hit Ratios: 


...

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
0,,2019-11-11 12:20:42,17.652 sec,0.0,0.833333,1.791759,0.806313
1,,2019-11-11 12:20:42,17.681 sec,1.0,0.761586,1.441517,0.127847
2,,2019-11-11 12:20:42,17.707 sec,2.0,0.698646,1.215178,0.114994
3,,2019-11-11 12:20:42,17.729 sec,3.0,0.642396,1.048866,0.111387
4,,2019-11-11 12:20:42,17.749 sec,4.0,0.592087,0.919148,0.105975
5,,2019-11-11 12:20:42,17.770 sec,5.0,0.547864,0.816253,0.106426
6,,2019-11-11 12:20:42,17.786 sec,6.0,0.508222,0.729874,0.104622
7,,2019-11-11 12:20:42,17.801 sec,7.0,0.473356,0.657732,0.101466
8,,2019-11-11 12:20:42,17.821 sec,8.0,0.442903,0.596940,0.094701
9,,2019-11-11 12:20:42,17.841 sec,9.0,0.415971,0.544306,0.095378



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,C17,2636.680420,1.000000,0.190932
1,C22,1897.120728,0.719511,0.137378
2,C20,1485.083740,0.563240,0.107541
3,C18,1168.854004,0.443305,0.084641
4,C34,1040.686523,0.394696,0.075360
5,C16,600.934082,0.227913,0.043516
6,C10,574.891174,0.218036,0.041630
7,C24,369.264313,0.140049,0.026740
8,C30,342.390656,0.129857,0.024794
9,C33,289.765717,0.109898,0.020983



See the whole table with table.as_data_frame()


<bound method H2OMultinomialModel.confusion_matrix of >